In [ ]:
import sys
sys.path.append('RenderMan/Builds/LinuxMakefile/build')
import librenderman as rm
import RenderPatches as rp
import FormatConverter as fc

import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
def play_audio(audio, rate=44100, save_fname=None):
    ipd.display(ipd.Audio(audio, rate=rate))
    if save_fname is not None:
        np.save(save_fname, audio)
    
import matplotlib.pyplot as plt
import numpy as np

def load_data(fname):
    npzfile = np.load(fname)
    return npzfile[npzfile.files[0]], npzfile[npzfile.files[1]] # presets, maxs
fname = "data_extraction/presets_and_maxs.npz"


In [ ]:
engine = rp.create_engine("Dexed.so")

# load presets
data, maxs = load_data(fname)
np.random.shuffle(data)

# render and save randoms
for i in range(40):
    audio,_ = rp.render_random(engine, True)
    play_audio(audio, save_fname="xp/random/"+str(i))

# render and save presets
for i in range(80):
    patch_data = fc.syx_to_rm(data[i])
    audio = rp.render_patch(patch_data, engine, override=True)
    play_audio(audio, save_fname="xp/real/"+str(i))
    